In [2]:
import pandas as pd
import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, cross_val_score
import numpy as np
from tpot import TPOTClassifier
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import balanced_accuracy_score
import joblib  # 用于保存交互项模式和模型

In [3]:
X_y_group_train = pd.read_csv('mid_data/X_y_group_train_updated_v8.7_线性回归v7&岭回归v2.csv')

print("Adding numeric labels y")
le = LabelEncoder()
X_y_group_train["y"] = le.fit_transform(X_y_group_train["label"])
# reordering columns:
X_y_group_train = X_y_group_train[["dataset", "variable"] + X_y_group_train.columns.drop(["dataset", "variable", "label", "y"]).tolist() + ["label", "y"]]


blacklist = ["ttest(v,X)", "pvalue(ttest(v,X))<=0.05", "ttest(v,Y)", "pvalue(ttest(v,Y))<=0.05", "ttest(X,Y)", "pvalue(ttest(X,Y))<=0.05"]
columns_to_drop = [col for col in blacklist if col in X_y_group_train.columns]
X_y_group_train = X_y_group_train.drop(columns=columns_to_drop)

display(X_y_group_train)

print("Extracting X_train, y_train, and group")
X_train = X_y_group_train.drop(["variable", "dataset", "label", "y"], axis="columns")
y_train = X_y_group_train["y"]
group_train = X_y_group_train["dataset"]

Adding numeric labels y


,dataset,variable,dimension,"corr(v,X)","corr(v,Y)","max(corr(v, others))","min(corr(v, others))","mean(corr(v, others))","std(corr(v, others))","corr(X,Y)",...,v~X_coefficient,v_squared~X_coefficient,v_cos~X_coefficient,v_sin~X_coefficient,X~Y_coefficient,v~Y_ridge_coefficient,v~X_ridge_coefficient,X~Y_ridge_coefficient,label,y
0,0,0,8,0.169735,-0.113595,0.791467,0.027355,0.282376,0.298969,-0.771058,...,2.581106,0.642049,0.627756,-1.853991,-0.243933,-0.270012,0.746844,-0.210404,Consequence of Y,5
1,0,2,8,-0.072334,0.139419,0.230139,0.003233,0.090511,0.082201,-0.771058,...,-0.057149,-0.355636,-0.302878,0.029001,-0.243933,-0.004777,-0.015475,-0.210404,Independent,6
2,0,3,8,0.123115,-0.238769,0.781051,0.012225,0.257037,0.288165,-0.771058,...,0.043431,-0.408441,-0.632885,-0.423769,-0.243933,0.087481,-0.235369,-0.210404,Cause of Y,1
3,0,4,8,-0.001935,0.013921,0.147408,0.001935,0.052451,0.053457,-0.771058,...,-0.055143,0.381220,0.427163,0.116956,-0.243933,-0.016076,0.045268,-0.210404,Cause of Y,1
4,0,5,8,0.284323,-0.466570,0.791467,0.064815,0.426306,0.250048,-0.771058,...,3.443463,-0.992699,-0.797171,-2.181048,-0.243933,-0.454817,1.085383,-0.210404,Mediator,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142905,46997,6,7,0.014740,0.697089,0.748635,0.013858,0.376630,0.352655,-0.089641,...,-0.414477,2.767481,2.688398,0.383910,-0.048813,0.046537,0.000815,-0.049793,Cause of Y,1
142906,46997,7,7,-0.007397,-0.116481,0.998843,0.007397,0.180741,0.333308,-0.089641,...,3.145600,1.175395,-5.843335,-3.135427,-0.048813,-0.015032,-0.000532,-0.049793,Cause of Y,1
142907,46997,8,7,-0.011658,0.908029,0.970364,0.011658,0.469777,0.441105,-0.089641,...,0.553463,2.599347,2.702726,-0.380755,-0.048813,0.499969,-0.000334,-0.049793,Cause of Y,1
142908,46998,0,2,0.083546,-0.019665,0.083546,0.016856,0.040022,0.037719,0.036862,...,0.449761,0.398762,0.421113,-0.405605,0.109254,-0.010355,0.044703,0.120088,Consequence of X,4


Extracting X_train, y_train, and group


In [4]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=13,
    n_jobs=-1,
    class_weight="balanced",
    random_state=42
)
display(model)

cv = GroupKFold(n_splits=4)
results = cross_val_score(
    model,
    X_train,
    y_train,
    groups=group_train,
    cv=cv,
    verbose=True,
    scoring="balanced_accuracy"
)

print(f"multiclass balanced accuracy: mean={results.mean()}")

RandomForestClassifier(class_weight='balanced', max_depth=13, n_jobs=-1,
                       random_state=42)

multiclass balanced accuracy: mean=0.4890766255617816


In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


# 提取特征和标签
X = X_y_group_train.drop(["variable", "dataset", "label", "y"], axis="columns")
y = X_y_group_train["y"]
groups = X_y_group_train["dataset"]

# 生成交互项
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X)

# 获取新特征的名称
feature_names = poly.get_feature_names_out(X.columns)

# 转换为DataFrame
X_poly_df = pd.DataFrame(X_poly, columns=feature_names)

scaler = StandardScaler()
X_poly_scaled = scaler.fit_transform(X_poly_df)

# 使用带有 L1 正则化的 LogisticRegressionCV 进行特征选择
logistic = LogisticRegressionCV(
    cv=5,
    penalty='l1',
    solver='saga',
    multi_class='multinomial',
    random_state=42,
    n_jobs=-1,
    max_iter=1000
)
logistic.fit(X_poly_scaled, y)

# 选择非零系数的特征
model = SelectFromModel(logistic, prefit=True)
X_selected = model.transform(X_poly_scaled)
selected_features = X_poly_df.columns[model.get_support()]
X_selected_df = pd.DataFrame(X_selected, columns=selected_features)

# 重置分组索引
group_train = groups.reset_index(drop=True)



In [ ]:
# 旧代码，提升约一个点

from sklearn.preprocessing import PolynomialFeatures

# 提取特征矩阵
X = X_y_group_train.drop(['variable', 'dataset', 'label', 'y'], axis=1)

# 使用随机森林获取特征重要性
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y_train)
importances = model.feature_importances_

# 将特征重要性与特征名称对应
feature_importance = pd.Series(importances, index=X.columns)
# 选择重要性排名前10的特征
top_features = feature_importance.nlargest(10).index.tolist()

# 仅对这些特征生成交互项
X_top = X[top_features]

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions_top = poly.fit_transform(X_top)
feature_names_top = poly.get_feature_names_out(input_features=top_features)
X_interactions_top_df = pd.DataFrame(X_interactions_top, columns=feature_names_top)

# 将新特征与原始特征合并
X_train_new = pd.concat([X.reset_index(drop=True), X_interactions_top_df], axis=1)



In [ ]:


# 继续使用新的特征矩阵进行模型训练
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=13,
    n_jobs=-1,
    class_weight='balanced',
    random_state=42
)

# 使用交叉验证评估模型性能
cv = GroupKFold(n_splits=4)
results = cross_val_score(
    model,
    X_train_new,
    y_train,
    groups=X_y_group_train['dataset'],
    cv=cv,
    verbose=True,
    scoring='balanced_accuracy'
)

print(f"Multiclass balanced accuracy with feature interactions: mean={results.mean()}")


旧分数:
0.47268